In [1]:
%matplotlib inline

In [2]:
from gammapy.maps import HpxMap, MapAxis
import numpy as np
from astropy.coordinates import SkyCoord
from astropy import units as u
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from gammapy.stats import cash

In [3]:
axis = MapAxis.from_nodes([0, 1, 2, 3], name='nhit')

In [4]:
counts = HpxMap.create(axes=[axis], nside=256, nest=False)
counts.data += 10
counts.data = np.random.poisson(counts.data)

background = HpxMap.create(axes=[axis], nside=256)
background.data += 5

exposure = HpxMap.create(axes=[axis], nside=256)
exposure.data += 0.0001

In [5]:
def f(f, counts, background, exposure):
    npred = background + f * exposure
    return cash(counts, npred).sum()

def extract_circular_region(hpx_map, center, radius, func=np.sum):
    coords = hpx_map.geom.to_image().get_coord()
    selection = coords.skycoord.separation(center) < radius
    return func(hpx_map.data[:, selection], axis=1)    

In [6]:
center = SkyCoord("0 deg", "0 deg")
radius = 2 * u.deg

counts_region = extract_circular_region(counts, center, radius)
background_region = extract_circular_region(background, center, radius)
exposure_region = extract_circular_region(exposure, center, radius, func=np.mean)

In [7]:
%%time
minimize(fun=f, x0=5, args=(counts_region, background_region, exposure_region))

CPU times: user 754 µs, sys: 484 µs, total: 1.24 ms
Wall time: 784 µs


      fun: -133209.55870364944
 hess_inv: array([[1]])
      jac: array([0.])
  message: 'Optimization terminated successfully.'
     nfev: 6
      nit: 1
     njev: 2
   status: 0
  success: True
        x: array([5.00195312])